In [12]:
import sys

sys.path.append( '/home/sedley/Work/FuzzingBook/fuzzingbook/notebooks/' )

import bookutils
from typing import Dict, Tuple, Union, List, Any

In [13]:
from fuzzingbook.Fuzzer import Fuzzer
from fuzzingbook.Fuzzer import Runner

In [14]:
import Intro_Testing

In [15]:
import random

In [16]:
def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    """A string of up to `max_length` characters
       in the range [`char_start`, `char_start` + `char_range`)"""
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [17]:
fuzzer()

'!7#%"*#0=)$;%6*;>638:*>80"=</>(/*:-(2<4 !:5*6856&?""11<7+%<%7,4.8,*+&,,$,."'

In [18]:
fuzzer(1, ord('a'), 26)

's'

In [19]:
fuzzer(100, ord('0'), 10)

'7185579035'

In [20]:
import os
import tempfile

basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

/tmp/tmppzqwqig8/input.txt


In [21]:
data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)

In [22]:
contents = open(FILE).read()
print(contents)
assert(contents == data)

 7:,>((/$$-/->.;.=;(.%!:50#7*8=$&&=$9!%6(4=&69':'<3+0-3.24#7=!&60)2/+";+<7+1<2!4$>92+$1


In [23]:
import subprocess

program = "bc"
with open(FILE, "w") as f:
    f.write("2 + 2\n")
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7

In [24]:
result.stdout

'4\n'

In [25]:
result.returncode

0

In [26]:
result.stderr

''

In [27]:
trials = 100
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
    runs.append((data, result))

In [28]:
sum(1 for (data, result) in runs if result.stderr == "")

7

In [29]:
errors = [(data, result) for (data, result) in runs if result.stderr != ""]
(first_data, first_result) = errors[0]

print(repr(first_data))
print(first_result.stderr)

'<(3%&5\'\'>#28($37"0*80!#\';1/ -6%=:80;!\'% 3&$;*27%%\'8&+$)0(?,-)>3=( #383\'7'
/tmp/tmppzqwqig8/input.txt 1: syntax error
/tmp/tmppzqwqig8/input.txt 1: syntax error
/tmp/tmppzqwqig8/input.txt 1: illegal character: '
/tmp/tmppzqwqig8/input.txt 1: illegal character: '



In [30]:
[result.stderr for (data, result) in runs if
 result.stderr != ""
 and "illegal character" not in result.stderr
 and "parse error" not in result.stderr
 and "syntax error" not in result.stderr]

[]

In [31]:
sum(1 for (data, result) in runs if result.returncode != 0)

0

In [32]:
def crash_if_too_long(s):
    buffer = "Thursday"
    if len(s) > len(buffer):
        raise ValueError

In [33]:
from ExpectError import ExpectError

trials = 100
with ExpectError():
    for i in range(trials):
        s = fuzzer()
        crash_if_too_long(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/1500401095.py", line 7, in <module>
    crash_if_too_long(s)
  File "/tmp/ipykernel_11966/2784561514.py", line 4, in crash_if_too_long
    raise ValueError
ValueError (expected)


In [34]:
def hang_if_no_space(s):
    i = 0
    while True:
        if i < len(s):
            if s[i] == ' ':
                break
        i += 1

In [35]:
from ExpectError import ExpectTimeout

trials = 100
with ExpectTimeout(2):
    for i in range(trials):
        s = fuzzer()
        hang_if_no_space(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/3845093739.py", line 7, in <module>
    hang_if_no_space(s)
  File "/tmp/ipykernel_11966/3035466707.py", line 4, in hang_if_no_space
    if i < len(s):
  File "/home/sedley/Work/FuzzingBook/fuzzingbook/notebooks/Timeout.ipynb", line 43, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)


In [36]:
def collapse_if_too_large(s):
    if int(s) > 1000:
        raise ValueError

In [37]:
long_number = fuzzer(100, ord('0'), 10)
print(long_number)

97531157058673604568813115


In [38]:
with ExpectError():
    collapse_if_too_large(long_number)

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/2775103647.py", line 2, in <module>
    collapse_if_too_large(long_number)
  File "/tmp/ipykernel_11966/1591744602.py", line 3, in collapse_if_too_large
    raise ValueError
ValueError (expected)


In [39]:
with open("program.c", "w") as f:
    f.write("""
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """)

In [40]:
!clang -fsanitize=address -g -o program program.c

In [41]:
!./program 99; echo $?

42


In [42]:
!./program 110

==89049==ERROR: AddressSanitizer: heap-buffer-overflow on address 0x60b00000015e at pc 0x559de31eff4c bp 0x7ffea60cc7d0 sp 0x7ffea60cc7c8
READ of size 1 at 0x60b00000015e thread T0
    #0 0x559de31eff4b in main /home/sedley/Work/FuzzingBook/program.c:12:16
    #1 0x7f1921229d8f in __libc_start_call_main csu/../sysdeps/nptl/libc_start_call_main.h:58:16
    #2 0x7f1921229e3f in __libc_start_main csu/../csu/libc-start.c:392:3
    #3 0x559de3132304 in _start (/home/sedley/Work/FuzzingBook/program+0x1e304) (BuildId: 09a02a8b8f7226bc9f4ff66da0504fa49f4282df)

0x60b00000015e is located 10 bytes to the right of 100-byte region [0x60b0000000f0,0x60b000000154)
allocated by thread T0 here:
    #0 0x559de31b514e in __interceptor_malloc (/home/sedley/Work/FuzzingBook/program+0xa114e) (BuildId: 09a02a8b8f7226bc9f4ff66da0504fa49f4282df)
    #1 0x559de31efebf in main /home/sedley/Work/FuzzingBook/program.c:7:17
    #2 0x7f1921229d8f in __libc_start_call_main csu/../sysdeps/nptl/libc_start_call_main.h:

In [43]:
!rm -fr program program.*

In [44]:
secrets = ("<space for reply>" + fuzzer(100) +
           "<secret-certificate>" + fuzzer(100) +
           "<secret-key>" + fuzzer(100) + "<other-secrets>")

In [45]:
uninitialized_memory_marker = "deadbeef"
while len(secrets) < 2048:
    secrets += uninitialized_memory_marker


In [46]:
def heartbeat(reply: str, length: int, memory: str) -> str:
    # Store reply in memory
    memory = reply + memory[len(reply):]

    # Send back heartbeat
    s = ""
    for i in range(length):
        s += memory[i]
    return s

In [47]:
heartbeat("potato", 6, memory=secrets)

'potato'

In [48]:
heartbeat("bird", 4, memory=secrets)

'bird'

In [49]:
heartbeat("hat", 500, memory=secrets)

'hatace for reply>/$,0\'<><5650;5;)4()*#/*",36.0"4" 07#88=&=; 27\'*-6 <secret-certificate>:,=.,):=&4=-.4-;:8? %>8!"$7/7)86\'+*2*& 875#.(/; :;=>5 ?32/!>31)9!<secret-key>446;*";;*(?%"6,;9=3.)62, 0\'11\'7.,6%./ <3:,2!/7<other-secrets>deadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdea'

In [50]:
from ExpectError import ExpectError

with ExpectError():
    for i in range(10):
        s = heartbeat(fuzzer(), random.randint(1, 500), memory=secrets)
        assert not s.find(uninitialized_memory_marker)
        assert not s.find("secret")

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/2459562132.py", line 6, in <module>
    assert not s.find(uninitialized_memory_marker)
AssertionError (expected)


In [51]:
airport_codes: Dict[str, str] = {
    "YVR": "Vancouver",
    "JFK": "New York-JFK",
    "CDG": "Paris-Charles de Gaulle",
    "CAI": "Cairo",
    "LED": "St. Petersburg",
    "PEK": "Beijing",
    "HND": "Tokyo-Haneda",
    "AKL": "Auckland"
}  # plus many more

airport_codes["YVR"]

"AKL" in airport_codes

True

In [52]:
def code_repOK(code: str) -> bool:
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
    for c in code:
        assert c.isalpha(), "Non-letter in airport code: " + repr(code)
        assert c.isupper(), "Lowercase letter in airport code: " + repr(code)
    return True

assert code_repOK("SEA")

In [53]:
def airport_codes_repOK():
    for code in airport_codes:
        assert code_repOK(code)
    return True

with ExpectError():
    assert airport_codes_repOK()

In [54]:
airport_codes["YMML"] = "Melbourne"

with ExpectError():
    assert airport_codes_repOK()

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/869960518.py", line 4, in <module>
    assert airport_codes_repOK()
  File "/tmp/ipykernel_11966/394991274.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
  File "/tmp/ipykernel_11966/1386186712.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'YMML' (expected)


In [55]:
def add_new_airport(code: str, city: str) -> None:
    assert code_repOK(code)
    airport_codes[code] = city

with ExpectError():  # For BER, ExpectTimeout would be more appropriate
    add_new_airport("BER", "Berlin")

In [56]:
with ExpectError():
    add_new_airport("London-Heathrow", "LHR")

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/1427835309.py", line 2, in <module>
    add_new_airport("London-Heathrow", "LHR")
  File "/tmp/ipykernel_11966/3789869367.py", line 2, in add_new_airport
    assert code_repOK(code)
  File "/tmp/ipykernel_11966/1386186712.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'London-Heathrow' (expected)


In [57]:
def add_new_airport_2(code: str, city: str) -> None:
    assert code_repOK(code)
    assert airport_codes_repOK()
    airport_codes[code] = city
    assert airport_codes_repOK()

In [58]:
with ExpectError():
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")

Traceback (most recent call last):
  File "/tmp/ipykernel_11966/1096579022.py", line 2, in <module>
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")
  File "/tmp/ipykernel_11966/2099116665.py", line 3, in add_new_airport_2
    assert airport_codes_repOK()
  File "/tmp/ipykernel_11966/394991274.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
  File "/tmp/ipykernel_11966/1386186712.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'YMML' (expected)


In [59]:
class RedBlackTree:
    def repOK(self):
        assert self.rootHasNoParent()
        assert self.rootIsBlack()
        assert self.rootNodesHaveOnlyBlackChildren()
        assert self.treeIsAcyclic()
        assert self.parentsAreConsistent()
        return True

    def rootIsBlack(self):
        if self.parent is None:
            assert self.color == BLACK
        return True

    def add_element(self, elem):
        assert self.repOK()
        ...  # Add the element
        assert self.repOK()

    def delete_element(self, elem):
        assert self.repOK()
        ...  # Delete the element
        assert self.repOK()

In [60]:
Outcome = str

class Runner:
    """Base class for testing inputs."""

    # Test outcomes
    PASS = "PASS"
    FAIL = "FAIL"
    UNRESOLVED = "UNRESOLVED"

    def __init__(self) -> None:
        """Initialize"""
        pass

    def run(self, inp: str) -> Any:
        """Run the runner with the given input"""
        return (inp, Runner.UNRESOLVED)

In [61]:
class PrintRunner(Runner):
    """Simple runner, printing the input."""

    def run(self, inp) -> Any:
        """Print the given input"""
        print(inp)
        return (inp, Runner.UNRESOLVED)

In [62]:
p = PrintRunner()
(result, outcome) = p.run("Some input")

Some input


In [63]:
result

'Some input'

In [64]:
outcome

'UNRESOLVED'

In [65]:
class ProgramRunner(Runner):
    """Test a program with inputs."""

    def __init__(self, program: Union[str, List[str]]) -> None:
        """Initialize.
           `program` is a program spec as passed to `subprocess.run()`"""
        self.program = program

    def run_process(self, inp: str = "") -> subprocess.CompletedProcess:
        """Run the program with `inp` as input.
           Return result of `subprocess.run()`."""
        return subprocess.run(self.program,
                              input=inp,
                              stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE,
                              universal_newlines=True)

    def run(self, inp: str = "") -> Tuple[subprocess.CompletedProcess, Outcome]:
        """Run the program with `inp` as input.  
           Return test outcome based on result of `subprocess.run()`."""
        result = self.run_process(inp)

        if result.returncode == 0:
            outcome = self.PASS
        elif result.returncode < 0:
            outcome = self.FAIL
        else:
            outcome = self.UNRESOLVED

        return (result, outcome)

In [66]:
class BinaryProgramRunner(ProgramRunner):
    def run_process(self, inp: str = "") -> subprocess.CompletedProcess:
        """Run the program with `inp` as input.  
           Return result of `subprocess.run()`."""
        return subprocess.run(self.program,
                              input=inp.encode(),
                              stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE)

In [67]:
cat = ProgramRunner(program="cat")
cat.run("hello")

(CompletedProcess(args='cat', returncode=0, stdout='hello', stderr=''), 'PASS')

In [68]:
class Fuzzer:
    """Base class for fuzzers."""

    def __init__(self) -> None:
        """Constructor"""
        pass

    def fuzz(self) -> str:
        """Return fuzz input"""
        return ""

    def run(self, runner: Runner = Runner()) \
            -> Tuple[subprocess.CompletedProcess, Outcome]:
        """Run `runner` with fuzz input"""
        return runner.run(self.fuzz())

    def runs(self, runner: Runner = PrintRunner(), trials: int = 10) \
            -> List[Tuple[subprocess.CompletedProcess, Outcome]]:
        """Run `runner` with fuzz input, `trials` times"""
        return [self.run(runner) for i in range(trials)]

In [69]:
class RandomFuzzer(Fuzzer):
    """Produce random inputs."""

    def __init__(self, min_length: int = 10, max_length: int = 100,
                 char_start: int = 32, char_range: int = 32) -> None:
        """Produce strings of `min_length` to `max_length` characters
           in the range [`char_start`, `char_start` + `char_range`)"""
        self.min_length = min_length
        self.max_length = max_length
        self.char_start = char_start
        self.char_range = char_range

    def fuzz(self) -> str:
        string_length = random.randrange(self.min_length, self.max_length + 1)
        out = ""
        for i in range(0, string_length):
            out += chr(random.randrange(self.char_start,
                                        self.char_start + self.char_range))
        return out

In [70]:
random_fuzzer = RandomFuzzer(min_length=20, max_length=20)
for i in range(10):
    print(random_fuzzer.fuzz())

:)6;%/(<!#-9*511<81;
,*%>1&3-7(.4"'*;362!
6??$-8"=*6 .> ("*)%;
'81/"60=?/4,"7:-;)"-
$.3(),$(17-$#266717=
7;3$1:>!40&:=-<24"==
5=0&5+91#$.5 07;#,,!
?30>#61)$4--8=<7)4 )
3/%,5+! "4)0?.9+?3()
<42?=?0=7(+/+((1)#/0


In [71]:
for i in range(10):
    inp = random_fuzzer.fuzz()
    result, outcome = cat.run(inp)
    assert result.stdout == inp
    assert outcome == Runner.PASS

In [72]:
random_fuzzer.run(cat)

(CompletedProcess(args='cat', returncode=0, stdout='%,#$>/51$2964>;)2417', stderr=''),
 'PASS')

In [73]:
random_fuzzer.runs(cat, 10)

[(CompletedProcess(args='cat', returncode=0, stdout='9"2&4907.. !7:&--"=$', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='\',7*>(5=5\'.!*+&>")6%', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout=')=,/5?:&5) ";.0!=6>3', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='>)=,)6&,?:!#2))- ?:)', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout="63'-4,)9#839%)?&(0<6", stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='"*;)!4?!(49+8=-\'&499', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout="?< '>>-*$ =(' #1.:3?", stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='?#7"&>;<-%8$:5+(?1/7', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='1&414;"-% !;&$3";/%6', stderr=''),
  'PASS'),
 (CompletedProcess(args='cat', returncode=0, stdout='31/06+</8/3&;0%*%822', stderr=''),
  'PASS')]

In [74]:
os.remove(FILE)
os.removedirs(tempdir)

In [146]:
def f1(s: str):
    for i in range(0, 32):
        if s.find(chr(i)) != -1 and s.find(chr(i)) == s.find("\D") + 2 and s.find("\D") != -1:
            raise ValueError

In [104]:
f1('\D')

In [105]:
s = 'asdf\D'
s += chr(0)
f1(s)

asdf\D 


ValueError: 

In [147]:
def f2(s: str):
    for i in range(128, 256):
        if s.find(chr(i)) != -1 and s.find(chr(i)) == s.find('\n') - 1 and s.find('\n') != -1:
            raise ValueError

In [79]:
s = 'Ç\n'
f2(s)

ValueError: 

In [148]:
def f3(s: str):
    if s.find('.') != -1 and s.find('.') == s.find('\n') - 1 and s.find('\n') != -1:
        raise ValueError

In [86]:
s = '.\n'
f3(s)

ValueError: 

In [150]:
class TroffRunner(Runner):
    """Troff Runner for the 2nd task."""
    def __init__(self) -> None:
        self.first_predicate_fails = 0
        self.second_predicate_fails = 0
        self.third_predicate_fails = 0

    def run(self, inp) -> Any:
        """Run with provided input all the constraint-functions."""
        try:
            f1(inp)
        except ValueError:
            self.first_predicate_fails += 1
            return (inp, Runner.FAIL)
        except Exception:
            return (inp, Runner.UNRESOLVED)
        
        try:
            f2(inp)
        except ValueError:
            self.second_predicate_fails += 1
            return (inp, Runner.FAIL)
        except Exception:
            return (inp, Runner.UNRESOLVED)
        
        try:
            f3(inp)
        except ValueError:
            self.third_predicate_fails += 1
            return (inp, Runner.FAIL)
        except Exception:
            return (inp, Runner.UNRESOLVED)
        
        return (inp, Runner.PASS)
    

In [152]:
random_fuzzer = RandomFuzzer(min_length=3, max_length=100, char_start = 0, char_range= 255)
troff_fuzzing = TroffRunner()
passes = 0
failes = 0
nan = 0

while (failes != 10000):
    (result, outcome) = troff_fuzzing.run(random_fuzzer.fuzz())
    if outcome == "FAIL":
        failes += 1
    elif outcome == "PASS":
        passes += 1
    elif outcome == "UNRESOLVED":
        nan += 1

print("The ratio of failes is " + str(failes / (failes + passes + nan)))
print("The ratio of unresolved cases is " + str(nan / (failes + passes + nan)))
print("The numbers of fails in the 1st, the 2nd and the 3rd predicate is " + 
     str(troff_fuzzing.first_predicate_fails) + " " + str(troff_fuzzing.second_predicate_fails)
      + " " + str(troff_fuzzing.third_predicate_fails))

The ratio of failes is 0.07774840615767377
The ratio of unresolved cases is 0.0
The numbers of fails in the 1st, the 2nd and the 3rd predicate is 7 9899 94


In [156]:
troff_bin = BinaryProgramRunner(program="troff")

passes = 0
failes = 0
nan = 0

while (failes != 50):
    (result, outcome) = troff_fuzzing.run(random_fuzzer.fuzz())
    if outcome == "FAIL":
        print(result, outcome)
        failes += 1
    elif outcome == "PASS":
        passes += 1
    elif outcome == "UNRESOLVED":
        nan += 1

print("The ratio of failes is " + str(failes / (failes + passes + nan)))
print("The ratio of unresolved cases is " + str(nan / (failes + passes + nan)))

9VeWï
ÙÍÛWVõ*ñí¸466É$[¢WïüE\®ã`oÌ2C`L<ÛÇò_³  FAIL
à@ÌÈ
#`Õ®²¡ã FAIL
Ñãí®ðã\«
}½6}Z\åøÜ¯u¡H# î fÀßÌlø}è.\Û'üopè)X,yê+Áb6ÎJðð@'  -JÑP¨þ	¶³ FAIL
5ãFl¸"EÛX1ð3u¦µ¿ì9À¨Y47Ó5yv
r5<mØ{y&¯Ý FAIL
¤|÷ùásuñ£ÚSø(¦[íÅ
 ýy¥¾(FÈä[æÚöÏ¼#Æ%rø^Ù¤ FAIL
à
â´i#ÁÀñD	ZüRª,Ñº¨P3$êÆûfüûÝ2ðxÄ#è×ÁH FAIL
ÁØtQ|gØãë¶k¥>õL	¶ðóÈOåCÆ
|PüiÙà$ÉQ¤-yÄá÷ªlF<hZtmznpç4 FAIL
!ö§â
 FAIL
PÈ¼¨¶S
øHË*Z©]Wwìý1|§r/*ü­Ï$¢ô^üÈ>úPè FAIL
µlr Y©ä °º
²î:o£dÈIÏíÄUPwëo£ØeÊåA?TÏXzÖ{K­iIPYÙ{û´Ç?8MÓÏ&?<®%ß  FAIL
¥UÆAÕ0ÄÏ´AIEäÝÈí6YnÕ[}[Üü·¤{û´Ïð6ìEYZb»
)ef]ñ±Z0Ä«¾JEà¾ FAIL
§{LGHÆ½¾]q:IGI= ¬hK¥ðo2µõp]¾x1¹>H¬Ñ»Ë¹$´læ
PìkR FAIL
c%®|·¹ì
ñ!*}'Ya:}ÂiþM:±õ>j[rðxfÌláã4= FAIL
OÎÊÉðåeÏº
ÕTøú^l£Ã"X5/¹O×ZA@`Öv,á`½tæµæ4ïáª@­ówötqéÎ FAIL
7"5Û	[³@Kè~kHKêöDn.¬íÔ9¡'iÖ
XÐ ÅÔÞ1 FAIL
äÂ¨f)iÑÏ£¤íä³Dºù¨úÔ
Ò°²¢m9òlk^¶¯UC.¡a/úºÑiºæ¡ FAIL
ø¿°³ÌÄà
°bØoéwí6;Q¥¨ç:Z±
,
O 8;Côö8º